In [1]:
import networkx as nx
import numpy as np
from torch_geometric.utils import to_networkx
import torch_geometric.data as pyg_data

from motclass import MotDataset, build_graph
from utilities import get_best_device

device = get_best_device()

# mot20_path = "/media/dmmp/vid+backup/Data/MOT20/images"
mot20_path = "data/MOT20"


[INFO] Using MPS.


In [2]:
def draw_graph(graph, track):
	G = to_networkx(graph, to_undirected=True)

	n_sum = [0] + list(np.cumsum(track.n_nodes))

	for i, n in enumerate(track.n_nodes):
		for j in range(n):
			G.nodes[j + n_sum[i]]['level'] = i

	pos = nx.multipartite_layout(G, subset_key="level")

	nx.draw(G, pos=pos, with_labels=True)


In [ ]:
mot20 = MotDataset(mot20_path, 'train', linkage_type="ADJACENT", device=device)

track = mot20[0]

adj_list, node_feats, frame_times, edge_attr = track.get_data()

del track


In [ ]:
graph = build_graph(adjacency_list=adj_list,
					flattened_node=node_feats,
					frame_times=frame_times,
					edge_partial_attributes=edge_attr,
					feature_extractor='resnet101',
					device=device)
